![Exit Icon](./icons/exit_128.png)

# Exit Strategy Explorer
## Interactively compare linear, logarithmic, and exponential scaling-out strategies.

The **Exit Strategy Explorer** provides a simple tool for investors to compare different exit strategies, namely linear, logarithmic, and exponential scaling-out strategies. Each strategy offers a unique approach to selling assets, and understanding the distinctions can significantly impact investment outcomes.

**Linear scaling-out strategy** involves selling a fixed quantity of an asset over a specified time period or price range. This method is straightforward and predictable, making it easy to implement and understand. Its main strength lies in its simplicity and consistent execution. However, it may not optimize returns during periods of high volatility, as it does not account for changes in market dynamics.

**Logarithmic scaling-out strategy** is designed to sell progressively smaller quantities of an asset as the price increases. This approach aims to capitalize on higher prices early on, securing profits while still retaining a portion of the asset for potential further increases. The strength of this strategy lies in its ability to balance risk and reward by taking advantage of rising markets. However, its complexity can be a drawback, as it requires more sophisticated decision-making and market analysis.

**Exponential scaling-out strategy** involves increasing the quantity of asset sales as prices rise. This aggressive strategy maximizes potential profits if the price continues to increase but can also lead to significant losses if the market reverses. The exponential method is best suited for markets experiencing rapid growth. The main strength of this strategy is its capacity to amplify profits during strong uptrends, but this comes at the cost of increased risk and potential for higher volatility in returns.

Each of these strategies offers distinct advantages and challenges, making it crucial for users to understand their underlying mechanisms and suitability for different market conditions.  
The **Exit Strategy Explorer** allows users to interactively simulate these strategies, providing valuable insights into their potential impacts on investment portfolios.

In [1]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def calculate_linear_scale_out(quantity, steps, price_points):
    """Calculates the amount to sell and the profits for linear scale-out."""
    amounts_to_sell = np.full(steps, quantity / steps)
    profits = amounts_to_sell * price_points
    return amounts_to_sell, profits

def calculate_exp_scale_out(quantity, steps, price_points):
    """Calculates the amount to sell and the profits for exponential scale-out."""
    exp_factors = np.linspace(0, 2, steps)
    exp_weights = np.exp(exp_factors)
    exp_weights_normalized = exp_weights / exp_weights.sum()
    amounts_to_sell = quantity * exp_weights_normalized
    profits = amounts_to_sell * price_points
    return amounts_to_sell, profits

def calculate_log_scale_out(quantity, steps, price_points):
    """Calculates the amount to sell and the profits for logarithmic scale-out."""
    log_factors = np.linspace(0.01, 1, steps)
    log_weights = np.log(log_factors + 1)
    log_weights_inverted = log_weights[::-1]  # Invert the array to make weights decrease
    log_weights_normalized = log_weights_inverted / log_weights_inverted.sum()  
    amounts_to_sell = quantity * log_weights_normalized
    profits = amounts_to_sell * price_points
    return amounts_to_sell, profits

def plot_strategy(df, title, price_points):
    """Plots the scale-out strategy."""
    fig, ax1 = plt.subplots(figsize=(14, 8))
    ax1.set_xlabel('Price ($)')
    ax1.set_xticks(df['Price ($)'])
    ax1.set_xticklabels([f'{price:.2f}' for price in df['Price ($)']], rotation=45, ha='right')

    # Disegno delle linee verticali per ogni x-datapoint
    for x in df['Price ($)']:
        ax1.axvline(x=x, linestyle='--', color='grey', alpha=0.5)  # Aggiungi linee verticali con un'opacità ridotta


    ax1.plot(df['Price ($)'], df['Compounding Profit ($)'], 'o-', color='red', label='Compounding Profit')
    ax1.set_ylabel('Compounding Profit ($)', color='red')
    ax1.tick_params(axis='y', labelcolor='red')
    # ax1.tick_params(axis='y', which='both', left=False, labelleft=False)

    ax2 = ax1.twinx()
    ax2.plot(df['Price ($)'], df['Compounding Sold Asset'], 'o--', color='blue', label='Compounding Sold Asset')
    ax2.set_ylabel('Compounding Sold Asset', color='blue')
    # ax2.tick_params(axis='y', labelcolor='blue')
    ax2.tick_params(axis='y', which='both', right=False, labelright=False)

    # Define the offsets
    offset_profit_x = 1 + (max(price_points) - min(price_points)) / (len(price_points) * 10)
    offset_profit_y = (max(df['Compounding Profit ($)']) - min(df['Compounding Profit ($)'])) * 0.02  # vertical offset for profit
    offset_asset_x = 1 + (max(price_points) - min(price_points)) / (len(price_points) * 10)
    offset_asset_y = (max(df['Compounding Sold Asset']) - min(df['Compounding Sold Asset'])) * 0.02  # vertical offset for assets

    # Annotating with the offsets
    for x, y in zip(df['Price ($)'], df['Compounding Profit ($)']):
        ax1.text(x + offset_profit_x, y - offset_profit_y, f"{y:.2f}", color='red', fontsize=8, ha='left')
    
    for x, y, z in zip(df['Price ($)'], df['Compounding Sold Asset'], df['Amount to Sell']):
        ax2.text(x - offset_asset_x, y + offset_asset_y, f"{z:.5f}\n({y:.3f})", color='blue', fontsize=8, ha='right')


    # Manually gathering the legend handles and labels from both axes
    handles, labels = [], []
    for ax in [ax1, ax2]:
        for handle, label in zip(*ax.get_legend_handles_labels()):
            handles.append(handle)
            labels.append(label)

    # Creating a single legend that includes information from both axes
    ax1.legend(handles, labels, loc='upper left')
    plt.title(title)
    plt.show()

def update_plot(quantity, steps, start_price, end_price, scale_method):
    """Updates the plot based on the selected parameters."""
    price_points = np.linspace(start_price, end_price, steps)
    
    if scale_method == 'Linear':
        amounts_to_sell, profits = calculate_linear_scale_out(quantity, steps, price_points)
    elif scale_method == 'Exp':
        amounts_to_sell, profits = calculate_exp_scale_out(quantity, steps, price_points)
    else:
        amounts_to_sell, profits = calculate_log_scale_out(quantity, steps, price_points)

    df = pd.DataFrame({
        'Price ($)': price_points,
        'Amount to Sell': amounts_to_sell,
        'Profits ($)': profits
    })
    
    df['Compounding Profit ($)'] = df['Profits ($)'].cumsum()
    df['Compounding Sold Asset'] = df['Amount to Sell'].cumsum()

    plot_strategy(df, f'{scale_method}-Out Strategy', price_points)

# Control Widgets
quantity_widget = widgets.FloatText(value=1.0, description='Quantity:')
steps_widget = widgets.IntSlider(value=10, min=3, max=30, step=1, description='Steps:')
start_price_widget = widgets.FloatText(value=80, description='Start Price:')
end_price_widget = widgets.FloatText(value=150, description='End Price:')
scale_method_widget = widgets.Dropdown(options=['Linear', 'Exp', 'Log'], value='Linear', description='Scale Method:')

ui = widgets.VBox([quantity_widget, steps_widget, start_price_widget, end_price_widget, scale_method_widget])
out = widgets.interactive_output(update_plot, {
    'quantity': quantity_widget,
    'steps': steps_widget,
    'start_price': start_price_widget,
    'end_price': end_price_widget,
    'scale_method': scale_method_widget
})

display(ui, out)

Output()